These scripts parses text looking for ability and skill checks and their DC.

In [54]:
import re, csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [55]:
skill_pattern = re.compile(r"\bDC\s+(\b\d+\b)\s+\b(Strength|Dexterity|Constitution|Wisdom|Intelligence|Charisma)\s+\((.+?)\)")

def find_skill_checks(filename):
    text = ""
    with open(filename, "r", encoding="utf-8") as textfile:
        text = ' '.join(textfile.readlines()).replace('\n', '')
    data = []
    for match in skill_pattern.finditer(text):
        d = {
            'DC': int(match.group(1)),
            'Ability': match.group(2),
        }
        try:
            d['Skill'] = match.group(3)
        except IndexError:
            d['Skill'] = None
        data.append(d)
    return data

save_pattern = re.compile(r"\bDC\s+(\b\d+\b)\s+\b(Strength|Dexterity|Constitution|Wisdom|Intelligence|Charisma)\s+?saving throw\b")

def find_saves(filename):
    text = ""
    with open(filename, "r", encoding="utf-8") as textfile:
        text = ' '.join(textfile.readlines()).replace('\n', '')
    data = []
    for match in save_pattern.finditer(text):
        d = {
            'DC': int(match.group(1)),
            'Ability': match.group(2),
        }
        data.append(d)
    return data

In [56]:
def prepare_data(data):
    dc_set = sorted(list(set([item['DC'] for item in data])))
    ability_set = sorted(list(set([item['Ability'] for item in data])))

    dc_count = {}
    for dc in dc_set:
        dc_count[dc] = [item['DC'] for item in data].count(dc)

    ability_count = {}
    for ability in ability_set:
        ability_count[ability] = [item['Ability'] for item in data].count(ability)

    ability_data = {}
    for ability in ability_set:
        ability_data[ability] = [item['DC'] for item in data if item['Ability'] == ability]
    
    return {
        'dc_set': dc_set,
        'ability_set': ability_set,
        'dc_count': dc_count, 
        'ability_count': ability_count, 
        'ability_data': ability_data}


In [74]:
wbtw_skills = find_skill_checks('dnd_data/rawmodules/wbtw.txt')

wbtw_saves = find_saves('dnd_data/rawmodules/wbtw.txt')
print(len(wbtw_saves))

skill_data = prepare_data(wbtw_skills)
save_data = prepare_data(wbtw_saves)

42


In [60]:
print(skill_data['ability_data'])

{'Charisma': [12, 15, 12, 14, 13, 12, 12, 15, 13, 12, 13, 15, 13, 17, 10, 13, 15, 15, 9, 9, 13, 12, 15, 12], 'Dexterity': [15, 12, 10, 13, 10, 15, 12, 13, 12, 16, 16, 14, 10, 12], 'Intelligence': [18, 15, 15, 13, 18, 15, 15, 10, 20, 20, 18, 15, 18, 20, 14, 15, 15, 15, 15, 15, 18, 16], 'Strength': [13, 10, 17, 17, 17, 13, 10, 16, 10, 25, 17, 15, 22, 30, 14, 16, 15, 22, 20, 20], 'Wisdom': [12, 14, 15, 15, 18, 12, 13, 15, 10, 10, 10, 10, 10, 15, 15, 16, 15, 12, 10, 16, 13, 12, 12, 17, 16, 15, 15, 15, 15, 15, 15]}


In [73]:
# with open('dnd_data/rawmodules/wbtw_skills.csv', 'w', newline='') as f:
#     writer = csv.writer(f)
#     writer.writerow(['Ability',] + skill_data['dc_set'])
#     for ability in skill_data['ability_set']:
#         ln = [ability,] + [skill_data['ability_data'][ability].count(dc) for dc in skill_data['dc_set']]
#         writer.writerow(ln)

# with open('dnd_data/rawmodules/wbtw_saves.csv', 'w', newline='') as f:
#     writer = csv.writer(f)
#     writer.writerow(['Ability',] + save_data['dc_set'])
#     for ability in save_data['ability_set']:
#         ln = [ability,] + [save_data['ability_data'][ability].count(dc) for dc in save_data['dc_set']]
#         writer.writerow(ln)
